In [15]:
import numpy as np
import pandas as pd

Read data

In [16]:
raw_data = pd.read_csv('./pull_requests_v3_December 12, 2019_16.35.csv') 
raw_data = raw_data.iloc[2:] # remove first two header rows

# rename columns
raw_data.rename(columns={
    "Q10": "gender",
    "Q11": "age",
    "Q12": "years_programming",
    "Q13": "years_javascript",
    "Q32": "num_crs_per_week",
    "Q14": "education",
    "Q19": "email",
    "Q17_1": "pct_men",
    "Q34": "employment",
    "Q45": "attn_app",
    "Q46": "attn_gender",
    "Q47": "attn_company",
    "Q48": "attn_name",
    "Q41": "code_quality_1",
    "Q42": "code_quality_2",
    "Version shown": "gender_shown",    
    
}, inplace=True)

In [17]:
# combine comments
max_num_comments = 20

col_to_index = {
    'line': 1,
    'type': 4,
    'text': 5
}

comment1 = 'Q43'
comment2 = 'Q50'

def get_keep_lambda(col1, col2, isnull_col1):
    def f(row):
        if row[isnull_col1]:
            return row[col2]
        else:
            return row[col1]
    return f

def merge_cols(col1, col2, new_col):
    null_col1 = 'TMP_null_col1'
    raw_data[null_col1] = raw_data[col1].isnull()
    raw_data[new_col] = raw_data.apply(get_keep_lambda(col1, col2, null_col1), axis=1)
    raw_data.drop([col1, col2, null_col1], axis=1, inplace=True)
        
for i in range(1,max_num_comments+1):
    for col, index in col_to_index.items():
        col1 = '{}_{}_{}'.format(comment1, i, index)
        col2 = '{}_{}_{}'.format(comment2, i, index)    
        new_col = 'comment_{}_{}'.format(i, col)
        merge_cols(col1, col2, new_col)
        
merge_cols('code_quality_1', 'code_quality_2', 'code_quality')


Get number of lines, types, comments

In [18]:
for col in col_to_index.keys():
    col_names = ['comment_{}_{}'.format(i, col) for i in range(1, max_num_comments+1)]
    raw_data['num_{}s'.format(col)] = max_num_comments - raw_data[col_names].isnull().sum(axis=1)
    
# get number of bugs v. quality
type_col_names = ['comment_{}_{}'.format(i, 'type') for i in range(1, max_num_comments+1)]
def count_bugs(row):
    num_bugs = 0
    for col in type_col_names:
        if type(row[col]) is float:
            continue
        if row[col].lower() == 'bug' or row[col].lower().startswith('b'):
            num_bugs += 1
    return num_bugs

def count_quality(row):
    num_quality = 0
    for col in type_col_names:
        if type(row[col]) is float:
            continue
        if row[col].lower() == 'quality' or row[col].lower().startswith('q'):
            num_quality += 1
    return num_quality

raw_data['num_bugs'] = raw_data.apply(count_bugs, axis=1)
raw_data['num_quality'] = raw_data.apply(count_quality, axis=1)
raw_data['percent_bugs'] = raw_data['num_bugs'] / raw_data['num_types']
raw_data['percent_quality'] = raw_data['num_quality'] / raw_data['num_types']
raw_data['gender_equal'] = raw_data['gender_shown'] == raw_data['gender']

In [22]:
raw_data['correct_app'] = raw_data['attn_app'] == 'Tip Calculator App'
raw_data['correct_company'] = raw_data['attn_company'] == 'Apps Inc'
raw_data['correct_gender'] = raw_data['attn_gender'] == raw_data['gender_shown']
def check_name_correct(row):
    if row['gender_shown'] == 'Female':
        return row['attn_name'] == 'Amy'
    else:
        return row['attn_name'] == 'Tom'
raw_data['correct_name'] = raw_data.apply(check_name_correct, axis=1)

Only keep rows that have 100% completion

In [23]:
complete_data = raw_data[raw_data['Progress'] == '100']
len(complete_data)

70

Strip identifying information

In [24]:
clean_data = complete_data.drop(['email', 'LocationLatitude', 'LocationLongitude', 'IPAddress',
                                 'RecipientFirstName', 'RecipientLastName', 'RecipientEmail'], axis=1)

In [25]:
clean_data.to_csv('./data_v3_processed.csv')